In [1]:
# Cell 1
!pip install transformers torch peft accelerate sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.9 MB/s eta 0:00:000:00:010:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 11.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 60.0 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.9.41
  Attempting uninstall: nvidia-curand-cu12
    Found existing installation: nvidia-curand-cu12 10.3.10.19
    Uninstalling nvidia-curand-cu12-

In [2]:
# Cell 2
# import unsloth # Remove
import torch
# from unsloth import FastLanguageModel # Remove
import pickle # Not used in the provided snippet, can be removed if not used elsewhere
import os # Not used in the provided snippet, can be removed if not used elsewhere

from transformers import AutoModelForCausalLM, AutoTokenizer 
from peft import PeftModel # Add
print(f"PyTorch version: {torch.__version__}")

2025-06-05 07:41:06.387630: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749109266.698088      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749109266.771841      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


PyTorch version: 2.6.0+cu124


In [3]:
# Cell 3 - (largely unchanged, ensure transformers is installed)
import os
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
import re
import sys

MODEL_ID = "aryan6637/ner_training_with_values"
ENTITY_TYPES = [
    "O",
    "B-SOIL_TYPE", "I-SOIL_TYPE",
    "B-NUTRIENT", "I-NUTRIENT",
    "B-TEMPERATURE_VALUE", "I-TEMPERATURE_VALUE",
    "B-HUMIDITY_VALUE", "I-HUMIDITY_VALUE",
    "B-MOISTURE_VALUE", "I-MOISTURE_VALUE",
    "B-NITROGEN_VALUE", "I-NITROGEN_VALUE",
    "B-POTASSIUM_VALUE", "I-POTASSIUM_VALUE",
    "B-PHOSPHOROUS_VALUE", "I-PHOSPHOROUS_VALUE"
]

LABEL_TO_ID = {label: i for i, label in enumerate(ENTITY_TYPES)}
ID_TO_LABEL = {i: label for label, i in LABEL_TO_ID.items()}

REQUIRED_PARAMS = {
    'Temparature': {'ner_tag': 'TEMPERATURE_VALUE', 'type': float, 'prompt': 'Enter Temperature (°C): '},
    'Humidity': {'ner_tag': 'HUMIDITY_VALUE', 'type': float, 'prompt': 'Enter Humidity (%): '},
    'Moisture': {'ner_tag': 'MOISTURE_VALUE', 'type': float, 'prompt': 'Enter Moisture (units): '},
    'Soil Type': {'ner_tag': 'SOIL_TYPE', 'type': str, 'prompt': 'Enter Soil Type: '},
    'Nitrogen': {'ner_tag': 'NITROGEN_VALUE', 'type': int, 'prompt': 'Enter Nitrogen (ppm): '},
    'Potassium': {'ner_tag': 'POTASSIUM_VALUE', 'type': int, 'prompt': 'Enter Potassium (ppm): '},
    'Phosphorous': {'ner_tag': 'PHOSPHOROUS_VALUE', 'type': int, 'prompt': 'Enter Phosphorous (ppm): '},
}

def clean_numeric_string(value_str):
    if value_str is None:
        return None
    return value_str.replace('%', '').replace('°C', '').replace('C', '').replace('units', '').replace('ppm', '').replace(',', '').strip()

def clean_soil_type_string(value_str):
    if value_str is None:
        return None
    return value_str.strip().rstrip(',.').strip()

def clean_nutrient_name_string(value_str):
    if value_str is None:
        return None
    return value_str.strip().rstrip(',.').strip()

tokenizer_bert = AutoTokenizer.from_pretrained(MODEL_ID)
bert_model = AutoModelForTokenClassification.from_pretrained(MODEL_ID) # Renamed 'bert' to 'bert_model' to avoid conflict
# Ensure pipeline uses CPU if no GPU is available. `device=-1` forces CPU.
# If you have a GPU but want to force CPU: device=-1
# Otherwise, it will auto-detect.
nlp_device = 0 if torch.cuda.is_available() else -1
nlp = pipeline("ner", model=bert_model, tokenizer=tokenizer_bert, aggregation_strategy="simple", device=nlp_device)
print(f"NER pipeline running on {'GPU' if nlp_device == 0 else 'CPU'}.")

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/669k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/431M [00:00<?, ?B/s]

Device set to use cpu


NER pipeline running on CPU.


In [4]:
# Cell 4
adapter_repo_id = "aryan6637/deepseek-crop-fertilizer-info-v3"
base_model_name = "deepseek-ai/deepseek-llm-7b-base" # Base model used for fine-tuning
max_seq_length = 1024

In [5]:
# Cell 5
print(f"Loading base model {base_model_name} for CPU...")
model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    torch_dtype=torch.float32,  # Use float32 for CPU; bfloat16 might work on some newer CPUs
    # device_map="auto" will try to use GPU if available, then CPU.
    # Forcing CPU:
    # device_map="cpu", # Or handle with .to('cpu') later
)

print(f"Loading tokenizer from {adapter_repo_id}...")
tokenizer = AutoTokenizer.from_pretrained(adapter_repo_id, trust_remote_code=True) # Some tokenizers need this

print(f"Loading adapter from {adapter_repo_id} and merging...")
# Load the LoRA adapter
model = PeftModel.from_pretrained(model, adapter_repo_id)
# Merge the adapter into the base model for potentially faster CPU inference
# This increases memory usage as it creates a full model.
model = model.merge_and_unload()

model.to("cpu") # Explicitly move to CPU
print("Model and tokenizer loaded successfully on CPU.")
print("WARNING: Running a 7B parameter model on CPU will be very slow for inference.")


# Handle tokenizer padding - Unsloth logs indicate <|PAD_TOKEN|> is used.
if tokenizer.pad_token is None or tokenizer.pad_token_id is None:
    print("Tokenizer does not have a pad token or pad_token_id. Adding '<|PAD_TOKEN|>'.")
    # Check if the token already exists to avoid adding duplicates if config is weird
    if '<|PAD_TOKEN|>' not in tokenizer.get_vocab():
        tokenizer.add_special_tokens({'pad_token': '<|PAD_TOKEN|>'})
    else:
        tokenizer.pad_token = '<|PAD_TOKEN|>'
    # After adding or setting, ensure pad_token_id is correct
    tokenizer.pad_token_id = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)

    # Resize model embeddings if new tokens were truly added (not just setting existing)
    # This is crucial if add_special_tokens actually modified the vocab size
    model.resize_token_embeddings(len(tokenizer))
    print(f"Resized model embeddings to: {len(tokenizer)}")


# Ensure the model's config pad_token_id is also updated to match the tokenizer
if model.config.pad_token_id is None or model.config.pad_token_id != tokenizer.pad_token_id:
    print(f"Updating model.config.pad_token_id to {tokenizer.pad_token_id}")
    model.config.pad_token_id = tokenizer.pad_token_id

print(f"Pad token: {tokenizer.pad_token}, ID: {tokenizer.pad_token_id}")
print(f"EOS token: {tokenizer.eos_token}, ID: {tokenizer.eos_token_id}")

Loading base model deepseek-ai/deepseek-llm-7b-base for CPU...


config.json:   0%|          | 0.00/584 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/22.5k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.85G [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/355 [00:00<?, ?B/s]

Loading adapter from aryan6637/deepseek-crop-fertilizer-info-v3 and merging...


adapter_config.json:   0%|          | 0.00/806 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/150M [00:00<?, ?B/s]

Model and tokenizer loaded successfully on CPU.
Updating model.config.pad_token_id to 100015
Pad token: <|PAD_TOKEN|>, ID: 100015
EOS token: <｜end▁of▁sentence｜>, ID: 100001


In [8]:

def get_parameters(sentence):
    if not sentence.strip():
        sys.exit(0)

    ner_results = nlp(sentence)
    extracted_raw_values = {}
    detected_nutrient_names = []

    for entity in ner_results:
        if entity['entity_group'] == 'NUTRIENT':
            cleaned_name = clean_nutrient_name_string(entity['word'])
            if cleaned_name and cleaned_name.lower() in ['nitrogen', 'n', 'potassium', 'k', 'phosphorous', 'phosphorus', 'p'] and cleaned_name not in detected_nutrient_names:
                detected_nutrient_names.append(cleaned_name)

        param_name_from_tag = None
        for req_param_name, details in REQUIRED_PARAMS.items():
            if details['ner_tag'] == entity['entity_group']:
                param_name_from_tag = req_param_name
                break

        if param_name_from_tag and param_name_from_tag not in extracted_raw_values:
            extracted_raw_values[param_name_from_tag] = entity['word']

    final_parameters = {}
    for param_name, details in REQUIRED_PARAMS.items():
        ner_tag = details['ner_tag']
        expected_type = details['type']
        prompt = details['prompt']
        raw_value_from_ner = extracted_raw_values.get(param_name)

        processed_value = None

        if raw_value_from_ner:
            cleaned_value_str = clean_numeric_string(raw_value_from_ner) if expected_type in [float, int] else clean_soil_type_string(raw_value_from_ner)
            try:
                if expected_type == float:
                    processed_value = float(cleaned_value_str)
                elif expected_type == int:
                    processed_value = int(float(cleaned_value_str))
                else:
                    processed_value = cleaned_value_str
            except ValueError:
                pass

        if processed_value is None:
            while True:
                manual_input_str = input(prompt)
                try:
                    if expected_type == float:
                        manual_value = float(manual_input_str)
                    elif expected_type == int:
                        manual_value = int(float(manual_input_str))
                    else:
                        manual_value = manual_input_str.strip()

                    if expected_type != str and manual_value < 0:
                        continue

                    processed_value = manual_value
                    break
                except ValueError:
                    continue

        final_parameters[param_name] = processed_value
    return final_parameters


In [6]:
# Cell 8
def get_prediction(final_parameters):
    inference_prompt = format_inference_prompt(final_parameters)
    # Tokenize the prompt
    device = "cpu" # Explicitly set to CPU
    inputs = tokenizer(inference_prompt, return_tensors="pt").to(device)

    print("\nGenerating response (this will be slow on CPU)...")
    with torch.no_grad(): # Ensure no gradients are calculated during inference
        outputs = model.generate(
            **inputs,
            max_new_tokens=150,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id, # Use the configured pad_token_id
            do_sample=True,
            temperature=0.6,
            top_p=0.9,
        )

    full_response_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print("\n--- Full Generated Text (with prompt) ---")
    print(full_response_text)

    # Extract only the newly generated part
    # The prompt itself might be slightly different after tokenization and decoding,
    # so find the response marker if possible, or slice carefully.
    # The model was trained with "### Response:\n"
    response_marker = "### Response:\n"
    # The inference_prompt ends with this marker.
    # We need the text *after* this marker in the *output*.
    # The output includes the input prompt.

    # A robust way to get only the generated part, assuming the prompt is part of outputs[0]:
    prompt_length_in_tokens = inputs.input_ids.shape[1]
    generated_tokens = outputs[0][prompt_length_in_tokens:]
    response_only_text = tokenizer.decode(generated_tokens, skip_special_tokens=True).strip()

    # Fallback if above is tricky or if prompt isn't perfectly echoed:
    # if response_marker in full_response_text:
    #     response_only_text = full_response_text.split(response_marker, 1)[-1].strip()
    # else: # A less robust fallback
    #     response_only_text = full_response_text[len(inference_prompt):].strip()


    print("\n--- Model's Recommendation ---")
    print(response_only_text)

In [9]:
def format_inference_prompt(final_parameters):
    instruction_text = (
        f"Given the following soil and environmental parameters:\n"
        f"- Temperature: {final_parameters['Temparature']}°C\n"
        f"- Humidity: {final_parameters['Humidity']}%\n"
        f"- Moisture: {final_parameters['Moisture']}\n"
        f"- Soil Type: {final_parameters['Soil Type']}\n"
        f"- Nitrogen: {final_parameters['Nitrogen']} ppm\n"
        f"- Potassium: {final_parameters['Potassium']} ppm\n"
        f"- Phosphorous: {final_parameters['Phosphorous']} ppm\n\n"
        f"Predict the suitable Crop Type and Fertilizer Name, and provide brief information about how they work or their characteristics."
    )
    # Alpaca format - the model was trained to generate text after "### Response:\n"
    formatted_prompt = f"### Instruction:\n{instruction_text}\n\n### Response:\n"
    return formatted_prompt

In [10]:
parameters = get_parameters(input("Enter the sentence"))

Enter the sentence The red soil contains 30 C with soil humidity of 45 % along with nutrient values N: 45 , P:34 , k:3
Enter Moisture (units):  12


In [11]:
parameters

{'Temparature': 30.0,
 'Humidity': 45.0,
 'Moisture': 12.0,
 'Soil Type': 'red',
 'Nitrogen': 45,
 'Potassium': 3,
 'Phosphorous': 34}

In [12]:
get_prediction(parameters)


Generating response (this will be slow on CPU)...

--- Full Generated Text (with prompt) ---
### Instruction:
Given the following soil and environmental parameters:
- Temperature: 30.0°C
- Humidity: 45.0%
- Moisture: 12.0
- Soil Type: red
- Nitrogen: 45 ppm
- Potassium: 3 ppm
- Phosphorous: 34 ppm

Predict the suitable Crop Type and Fertilizer Name, and provide brief information about how they work or their characteristics.

### Response:
Recommended Crop Type: Barley
Recommended Fertilizer: 20-20


--- Model's Recommendation ---
Recommended Crop Type: Barley
Recommended Fertilizer: 20-20
